In [19]:
import numpy as np
import pandas as pd

from tqdm import tqdm

In [20]:
main_dir = "competitive-data-science-predict-future-sales"

In [21]:
# Sample submission file
df = pd.read_csv(f"{main_dir}/sample_submission.csv")
df

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [45]:
X = np.load(f"{main_dir}/X.npy")
# X = np.load(f"{main_dir}/X_(1).npy")
# X = np.delete(X, 1, 1).astype('int')
print(X.shape)
X

(214200, 1, 33)


array([[[0, 0, 0, ..., 1, 3, 1]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 1, 3]],

       ...,

       [[1, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0]]])

In [23]:
Y = np.load(f"{main_dir}/Y.npy")
Y = Y.reshape((214200,1))
print(Y.shape)
Y

(214200, 1)


array([[0],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [28]:
# combine x and y for normalizing

def combine_x_y(x, y):
    ww = []
    ft = X.shape[1]
    for i in tqdm(range(len(x))):
      if ft == 5:
        ggi = [y[i][0], 0, x[i][2][0], x[i][3][0], x[i][4][0]]
      elif ft == 2: 
        ggi = [y[i][0], 0]
      elif ft == 1:
        ggi = [y[i][0]]
      ww.append(ggi)
    ww = np.array(ww, dtype='float32').reshape(214200, ft, 1)
    bg = np.concatenate([x, ww], axis=2)
    return bg

In [29]:
combined = combine_x_y(X, Y)
print(combined.shape)
combined

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 214200/214200 [00:00<00:00, 928590.76it/s]


(214200, 2, 34)


array([[[  0.        ,   0.        ,   0.        , ...,   3.        ,
           1.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ..., 999.16666667,
         749.5       ,   0.        ]],

       [[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        , ...,   1.        ,
           3.        ,   1.        ],
        [  0.        ,   0.        ,   0.        , ..., 599.        ,
         999.        ,   0.        ]],

       ...,

       [[  1.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [199.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
       

In [30]:
good_game = np.delete(combined, 0, 2)

In [31]:
# model_path = f"{main_dir}/model.h5"
# model_path = f"{main_dir}/model_2.h5"
model_path = f"{main_dir}/model_1.h5"

In [32]:
from keras.models import load_model

In [33]:
model = load_model(model_path)

In [34]:
gg = model.predict(good_game).round(0).astype('int').astype('float32')

   1/6694 [..............................] - ETA: 33:37

2022-10-15 12:05:19.951001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-15 12:05:19.987611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6694/6694 [==============================] - 14s 2ms/step


In [35]:
print(gg.shape)
gg

(214200, 1)


array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [36]:
np.unique(gg, return_counts=True)

(array([-8., -7., -5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.,  5.,
         6.,  7.,  8.,  9., 10., 11., 14., 15., 16.], dtype=float32),
 array([     8,      2,      5,      3,     15,    320,    129, 193354,
          8815,   3471,    810,    982,   5122,    127,    228,    201,
           260,    134,    190,      9,      5,     10]))

In [47]:
df['item_cnt_month'] = gg
df

,ID,item_cnt_month
0,0,7
1,1,0
2,2,7
3,3,0
4,4,0
...,...,...
214195,214195,7
214196,214196,0
214197,214197,0
214198,214198,0


In [48]:
sav_path = f"{main_dir}/result.csv"

In [49]:
df.to_csv(sav_path, index=False)

In [50]:
pd.read_csv(sav_path)

,ID,item_cnt_month
0,0,7
1,1,0
2,2,7
3,3,0
4,4,0
...,...,...
214195,214195,7
214196,214196,0
214197,214197,0
214198,214198,0
